# RANDOM FOREST

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats.mstats import winsorize
import scipy.stats as stats
from sklearn import linear_model
import statsmodels.api as sm
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score, mean_absolute_error,explained_variance_score
from sklearn.metrics import mean_squared_error as mse
from statsmodels.tools.eval_measures import mse, rmse
from sklearn.linear_model import Ridge
from sklearn.linear_model import Lasso
from sklearn.linear_model import ElasticNet
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.metrics import classification_report, precision_recall_fscore_support
from sklearn.metrics import roc_curve, roc_auc_score
from sklearn.metrics import precision_recall_curve
from sklearn.metrics import log_loss
from sklearn.utils import resample
from sklearn.model_selection import KFold, train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neighbors import KNeighborsRegressor
from sklearn.model_selection import cross_val_score
from sklearn.tree import DecisionTreeClassifier, plot_tree
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import RandomForestRegressor
from sklearn.svm import SVC
from sklearn.svm import SVR
import xgboost as xgb
import lightgbm as lgb

In [2]:
life_df=pd.read_csv("Life Expectancy Data.csv")
life_df.head()

,Country,Year,Status,Life expectancy,Adult Mortality,infant deaths,Alcohol,percentage expenditure,Hepatitis B,Measles,...,Polio,Total expenditure,Diphtheria,HIV/AIDS,GDP,Population,thinness 1-19 years,thinness 5-9 years,Income composition of resources,Schooling
0,Afghanistan,2015,Developing,65.0,263.0,62,0.01,71.279624,65.0,1154,...,6.0,8.16,65.0,0.1,584.259210,33736494.0,17.2,17.3,0.479,10.1
1,Afghanistan,2014,Developing,59.9,271.0,64,0.01,73.523582,62.0,492,...,58.0,8.18,62.0,0.1,612.696514,327582.0,17.5,17.5,0.476,10.0
2,Afghanistan,2013,Developing,59.9,268.0,66,0.01,73.219243,64.0,430,...,62.0,8.13,64.0,0.1,631.744976,31731688.0,17.7,17.7,0.470,9.9
3,Afghanistan,2012,Developing,59.5,272.0,69,0.01,78.184215,67.0,2787,...,67.0,8.52,67.0,0.1,669.959000,3696958.0,17.9,18.0,0.463,9.8
4,Afghanistan,2011,Developing,59.2,275.0,71,0.01,7.097109,68.0,3013,...,68.0,7.87,68.0,0.1,63.537231,2978599.0,18.2,18.2,0.454,9.5


In [3]:
life_df2=life_df.copy()

In [4]:
life_df2.columns.values

array(['Country', 'Year', 'Status', 'Life expectancy ', 'Adult Mortality',
       'infant deaths', 'Alcohol', 'percentage expenditure',
       'Hepatitis B', 'Measles ', ' BMI ', 'under-five deaths ', 'Polio',
       'Total expenditure', 'Diphtheria ', ' HIV/AIDS', 'GDP',
       'Population', ' thinness  1-19 years', ' thinness 5-9 years',
       'Income composition of resources', 'Schooling'], dtype=object)

Words have space

In [5]:
life_df2.columns = life_df2.columns.str.strip()

In [6]:
life_df2.columns.values

array(['Country', 'Year', 'Status', 'Life expectancy', 'Adult Mortality',
       'infant deaths', 'Alcohol', 'percentage expenditure',
       'Hepatitis B', 'Measles', 'BMI', 'under-five deaths', 'Polio',
       'Total expenditure', 'Diphtheria', 'HIV/AIDS', 'GDP', 'Population',
       'thinness  1-19 years', 'thinness 5-9 years',
       'Income composition of resources', 'Schooling'], dtype=object)

Removed the space

In [7]:
pd.get_dummies(life_df2["Status"])

,Developed,Developing
0,0,1
1,0,1
2,0,1
3,0,1
4,0,1
...,...,...
2933,0,1
2934,0,1
2935,0,1
2936,0,1


In [8]:
life_df2= pd.concat([life_df2, pd.get_dummies(life_df["Status"], drop_first=True)], axis=1)
life_df2.head()

,Country,Year,Status,Life expectancy,Adult Mortality,infant deaths,Alcohol,percentage expenditure,Hepatitis B,Measles,...,Total expenditure,Diphtheria,HIV/AIDS,GDP,Population,thinness 1-19 years,thinness 5-9 years,Income composition of resources,Schooling,Developing
0,Afghanistan,2015,Developing,65.0,263.0,62,0.01,71.279624,65.0,1154,...,8.16,65.0,0.1,584.259210,33736494.0,17.2,17.3,0.479,10.1,1
1,Afghanistan,2014,Developing,59.9,271.0,64,0.01,73.523582,62.0,492,...,8.18,62.0,0.1,612.696514,327582.0,17.5,17.5,0.476,10.0,1
2,Afghanistan,2013,Developing,59.9,268.0,66,0.01,73.219243,64.0,430,...,8.13,64.0,0.1,631.744976,31731688.0,17.7,17.7,0.470,9.9,1
3,Afghanistan,2012,Developing,59.5,272.0,69,0.01,78.184215,67.0,2787,...,8.52,67.0,0.1,669.959000,3696958.0,17.9,18.0,0.463,9.8,1
4,Afghanistan,2011,Developing,59.2,275.0,71,0.01,7.097109,68.0,3013,...,7.87,68.0,0.1,63.537231,2978599.0,18.2,18.2,0.454,9.5,1


## Missing Value

In [9]:
life_df2.isnull().sum()

Country                              0
Year                                 0
Status                               0
Life expectancy                     10
Adult Mortality                     10
infant deaths                        0
Alcohol                            194
percentage expenditure               0
Hepatitis B                        553
Measles                              0
BMI                                 34
under-five deaths                    0
Polio                               19
Total expenditure                  226
Diphtheria                          19
HIV/AIDS                             0
GDP                                448
Population                         652
thinness  1-19 years                34
thinness 5-9 years                  34
Income composition of resources    167
Schooling                          163
Developing                           0
dtype: int64

In [10]:
life_df3=life_df2.copy()

In [11]:
life_df3["Life expectancy"]=life_df3["Life expectancy"].fillna(life_df3.groupby("Country")["Life expectancy"].transform("mean"))
life_df3["Life expectancy"]=life_df3["Life expectancy"].fillna(life_df3["Life expectancy"].mean())

life_df3["Adult Mortality"]=life_df3["Adult Mortality"].fillna(life_df3.groupby("Country")["Adult Mortality"].transform("mean"))
life_df3["Adult Mortality"]=life_df3["Adult Mortality"].fillna(life_df3["Adult Mortality"].mean())

life_df3["Alcohol"]=life_df3["Alcohol"].fillna(life_df3.groupby("Country")["Alcohol"].transform("mean"))
life_df3["Alcohol"]=life_df3["Alcohol"].fillna(life_df3["Alcohol"].mean())

life_df3["Hepatitis B"]=life_df3["Hepatitis B"].fillna(life_df3.groupby("Country")["Hepatitis B"].transform("mean"))
life_df3["Hepatitis B"]=life_df3["Hepatitis B"].fillna(life_df3["Hepatitis B"].mean())

life_df3["BMI"]=life_df3["BMI"].fillna(life_df3.groupby("Country")["BMI"].transform("mean"))
life_df3["BMI"]=life_df3["BMI"].fillna(life_df3["BMI"].mean())

life_df3["Polio"]=life_df3["Polio"].fillna(life_df3.groupby("Country")["Polio"].transform("mean"))
life_df3["Polio"]=life_df3["Polio"].fillna(life_df3["Polio"].mean())

life_df3["Total expenditure"]=life_df3["Total expenditure"].fillna(life_df3.groupby("Country")["Total expenditure"].transform("mean"))
life_df3["Total expenditure"]=life_df3["Total expenditure"].fillna(life_df3["Total expenditure"].mean())

life_df3["Diphtheria"]=life_df3["Diphtheria"].fillna(life_df3.groupby("Country")["Diphtheria"].transform("mean"))
life_df3["Diphtheria"]=life_df3["Diphtheria"].fillna(life_df3["Diphtheria"].mean())

life_df3["GDP"]=life_df3["GDP"].fillna(life_df3.groupby("Country")["GDP"].transform("mean"))
life_df3["GDP"]=life_df3["GDP"].fillna(life_df3["GDP"].mean())

life_df3["Population"]=life_df3["Population"].fillna(life_df3.groupby("Country")["Population"].transform("mean"))
life_df3["Population"]=life_df3["Population"].fillna(life_df3["Population"].mean())

life_df3["thinness  1-19 years"]=life_df3["thinness  1-19 years"].fillna(life_df3.groupby("Country")["thinness  1-19 years"].transform("mean"))
life_df3["thinness  1-19 years"]=life_df3["thinness  1-19 years"].fillna(life_df3["thinness  1-19 years"].mean())

life_df3["thinness 5-9 years"]=life_df3["thinness 5-9 years"].fillna(life_df3.groupby("Country")["thinness 5-9 years"].transform("mean"))
life_df3["thinness 5-9 years"]=life_df3["thinness 5-9 years"].fillna(life_df3["thinness 5-9 years"].mean())

life_df3["Income composition of resources"]=life_df3["Income composition of resources"].fillna(life_df3.groupby("Country")["Income composition of resources"].transform("mean"))
life_df3["Income composition of resources"]=life_df3["Income composition of resources"].fillna(life_df3["Income composition of resources"].mean())

life_df3["Schooling"]=life_df3["Schooling"].fillna(life_df3.groupby("Country")["Schooling"].transform("mean"))
life_df3["Schooling"]=life_df3["Schooling"].fillna(life_df3["Schooling"].mean())


In [12]:
life_df3.isnull().sum()

Country                            0
Year                               0
Status                             0
Life expectancy                    0
Adult Mortality                    0
infant deaths                      0
Alcohol                            0
percentage expenditure             0
Hepatitis B                        0
Measles                            0
BMI                                0
under-five deaths                  0
Polio                              0
Total expenditure                  0
Diphtheria                         0
HIV/AIDS                           0
GDP                                0
Population                         0
thinness  1-19 years               0
thinness 5-9 years                 0
Income composition of resources    0
Schooling                          0
Developing                         0
dtype: int64

## 1. Classification

**Train Model**

In [13]:
y_rf = life_df3['Developing']
X_rf = life_df3.drop(['Developing', 'Year', 'Status', 'Country'], axis=1)

In [14]:
X_rf_train, X_rf_test, y_rf_train, y_rf_test =  train_test_split(X_rf, y_rf, test_size=0.20, random_state=111)

In [15]:
random_forest = RandomForestClassifier(n_estimators=25, random_state=111)
random_forest.fit(X_rf_train, y_rf_train) 

RandomForestClassifier(n_estimators=25, random_state=111)

**Prediciton**

In [ ]:
y_pred_rf_train = random_forest.predict(X_rf_train)
y_pred_rf_test = random_forest.predict(X_rf_test)

In [16]:
print("R-squared of the model in training set is: {}".format(random_forest.score(X_rf_train, y_rf_train)))
print("-----Test set statistics-----")
print("R-squared of the model in test set is: {}".format(random_forest.score(X_rf_test, y_rf_test)))


R-squared of the model in training set is: 1.0
-----Test set statistics-----
R-squared of the model in test set is: 0.9829931972789115


**KFold - Cross Validation**

**Train**

In [19]:
cvscores_2 = cross_val_score(random_forest, X_rf_train, y_rf_train, cv=2)
print('cv =  2 ---> Average cross validation score : ', np.mean(cvscores_2))

cvscores_3 = cross_val_score(random_forest, X_rf_train, y_rf_train, cv=3)
print('cv =  3 ---> Average cross validation score : ', np.mean(cvscores_3))

cvscores_4 = cross_val_score(random_forest, X_rf_train, y_rf_train, cv=4)
print('cv =  4 ---> Average cross validation score : ', np.mean(cvscores_4))

cvscores_5 = cross_val_score(random_forest, X_rf_train, y_rf_train, cv=5)
print('cv =  5 ---> Average cross validation score : ', np.mean(cvscores_5))

cvscores_6 = cross_val_score(random_forest, X_rf_train, y_rf_train, cv=6)
print('cv =  6 ---> Average cross validation score : ', np.mean(cvscores_6))

cvscores_7 = cross_val_score(random_forest, X_rf_train, y_rf_train, cv=15)
print('cv = 7 ---> Average cross validation score  : ', np.mean(cvscores_7))

cv =  2 ---> Average cross validation score :  0.9765957446808511
cv =  3 ---> Average cross validation score :  0.9804248008270996
cv =  4 ---> Average cross validation score :  0.9829787110755717
cv =  5 ---> Average cross validation score :  0.9825531914893617
cv =  6 ---> Average cross validation score :  0.9863815091253892
cv = 7 ---> Average cross validation score  :  0.9846834340464912


**Test**

In [20]:
cvscores_2 = cross_val_score(random_forest, X_rf_test, y_rf_test, cv=2)
print('cv =  2 ---> Average cross validation score : ', np.mean(cvscores_2))

cvscores_3 = cross_val_score(random_forest, X_rf_test, y_rf_test, cv=3)
print('cv =  3 ---> Average cross validation score : ', np.mean(cvscores_3))

cvscores_4 = cross_val_score(random_forest, X_rf_test, y_rf_test, cv=4)
print('cv =  4 ---> Average cross validation score : ', np.mean(cvscores_4))

cvscores_5 = cross_val_score(random_forest, X_rf_test, y_rf_test, cv=5)
print('cv =  5 ---> Average cross validation score : ', np.mean(cvscores_5))

cvscores_6 = cross_val_score(random_forest, X_rf_test, y_rf_test, cv=6)
print('cv =  6 ---> Average cross validation score : ', np.mean(cvscores_6))

cvscores_7 = cross_val_score(random_forest, X_rf_test, y_rf_test, cv=15)
print('cv = 7 ---> Average cross validation score  : ', np.mean(cvscores_7))

cv =  2 ---> Average cross validation score :  0.9404761904761905
cv =  3 ---> Average cross validation score :  0.9438775510204082
cv =  4 ---> Average cross validation score :  0.9455782312925171
cv =  5 ---> Average cross validation score :  0.9489787049109083
cv =  6 ---> Average cross validation score :  0.9523809523809524
cv = 7 ---> Average cross validation score  :  0.9473504273504274


## 2. Regression

**Train Model**

In [21]:
y_rf2 = life_df3['Life expectancy']
X_rf2 = life_df3.drop(['Life expectancy', 'Status', 'Country'], axis=1)

In [22]:
X_rf2_train, X_rf2_test, y_rf2_train, y_rf2_test =  train_test_split(X_rf2, y_rf2, test_size=0.20, random_state=111)

In [23]:
random_forest2 = RandomForestRegressor(n_estimators=25, random_state=111)
random_forest2.fit(X_rf2_train, y_rf2_train) 

RandomForestRegressor(n_estimators=25, random_state=111)

**Prediciton**

In [25]:
y_pred_rf2_train = random_forest2.predict(X_rf2_train)
y_pred_rf2_test = random_forest2.predict(X_rf2_test)

In [26]:
print("R-squared of the model in training set is: {}".format(random_forest2.score(X_rf2_train, y_rf2_train)))
print("-----Test set statistics-----")
print("R-squared of the model in test set is: {}".format(random_forest2.score(X_rf2_test, y_rf2_test)))
print("Mean absolute error (MAE) of the prediction is: {}".format(mean_absolute_error(y_rf2_test, y_pred_rf2_test)))
print("Mean squared error (MSE) of the prediction is: {}".format(mse(y_rf2_test, y_pred_rf2_test)))
print("Root mean squared error (RMSE) of the prediction is: {}".format(rmse(y_rf2_test, y_pred_rf2_test)))
print("Mean absolute percentage error (MAPE) of the prediction is: {}".format(np.mean(np.abs((y_rf2_test - y_pred_rf2_test) / y_rf2_test)) * 100))

R-squared of the model in training set is: 0.9940072164985907
-----Test set statistics-----
R-squared of the model in test set is: 0.9621797247233843
Mean absolute error (MAE) of the prediction is: 1.2012398284450398
Mean squared error (MSE) of the prediction is: 3.292451369154888
Root mean squared error (RMSE) of the prediction is: 1.8145113306769092
Mean absolute percentage error (MAPE) of the prediction is: 1.7896359372190596


**KFold - Cross Validation**

**Train**

In [27]:
cvscores_2 = cross_val_score(random_forest2, X_rf2_train, y_rf2_train, cv=2)
print('cv =  2 ---> Average cross validation score : ', np.mean(cvscores_2))

cvscores_3 = cross_val_score(random_forest2, X_rf2_train, y_rf2_train, cv=3)
print('cv =  3 ---> Average cross validation score : ', np.mean(cvscores_3))

cvscores_4 = cross_val_score(random_forest2, X_rf2_train, y_rf2_train, cv=4)
print('cv =  4 ---> Average cross validation score : ', np.mean(cvscores_4))

cvscores_5 = cross_val_score(random_forest2, X_rf2_train, y_rf2_train, cv=5)
print('cv =  5 ---> Average cross validation score : ', np.mean(cvscores_5))

cvscores_6 = cross_val_score(random_forest2, X_rf2_train, y_rf2_train, cv=6)
print('cv =  6 ---> Average cross validation score : ', np.mean(cvscores_6))

cvscores_7 = cross_val_score(random_forest2, X_rf2_train, y_rf2_train, cv=15)
print('cv = 7 ---> Average cross validation score  : ', np.mean(cvscores_7))

cv =  2 ---> Average cross validation score :  0.9446546485181064
cv =  3 ---> Average cross validation score :  0.9510918466016408
cv =  4 ---> Average cross validation score :  0.9552564531111712
cv =  5 ---> Average cross validation score :  0.9569788995078398
cv =  6 ---> Average cross validation score :  0.9585662498213187
cv = 7 ---> Average cross validation score  :  0.9589781879304528


**Test**

In [28]:
cvscores_2 = cross_val_score(random_forest2, X_rf2_test, y_rf2_test, cv=2)
print('cv =  2 ---> Average cross validation score : ', np.mean(cvscores_2))

cvscores_3 = cross_val_score(random_forest2, X_rf2_test, y_rf2_test, cv=3)
print('cv =  3 ---> Average cross validation score : ', np.mean(cvscores_3))

cvscores_4 = cross_val_score(random_forest2, X_rf2_test, y_rf2_test, cv=4)
print('cv =  4 ---> Average cross validation score : ', np.mean(cvscores_4))

cvscores_5 = cross_val_score(random_forest2, X_rf2_test, y_rf2_test, cv=5)
print('cv =  5 ---> Average cross validation score : ', np.mean(cvscores_5))

cvscores_6 = cross_val_score(random_forest2, X_rf2_test, y_rf2_test, cv=6)
print('cv =  6 ---> Average cross validation score : ', np.mean(cvscores_6))

cvscores_7 = cross_val_score(random_forest2, X_rf2_test, y_rf2_test, cv=15)
print('cv = 7 ---> Average cross validation score  : ', np.mean(cvscores_7))

cv =  2 ---> Average cross validation score :  0.8950010446558947
cv =  3 ---> Average cross validation score :  0.9173793295050502
cv =  4 ---> Average cross validation score :  0.9127565702834729
cv =  5 ---> Average cross validation score :  0.9209362100268426
cv =  6 ---> Average cross validation score :  0.9170217122389782
cv = 7 ---> Average cross validation score  :  0.9186029120368503
